# Project 3: Linear Regression
File "wine.csv" là cơ sở dữ liệu đánh giá chất lượng của 1200 chai rượu vang theo thang điểm 1 - 10 dựa trên 11 tính chất khác nhau. <br>
Xây dựng mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính.


## A. Sử dụng toàn bộ 11 đặc trưng đề bài cung cấp

In [2]:
import pandas as pd
# Đường dẫn đến file csv
csvPath = 'https://raw.githubusercontent.com/tarzanchemgio/Project-Linear-Regression/master/dataset/wine.csv?token=ALHLG3TXRNZZS76X4VICHTK7KCVAM'
df = pd.read_csv(csvPath, sep=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1194,7.0,0.745,0.12,1.8,0.114,15.0,64,0.99588,3.22,0.59,9.5,6
1195,6.2,0.430,0.22,1.8,0.078,21.0,56,0.99633,3.52,0.60,9.5,6
1196,7.9,0.580,0.23,2.3,0.076,23.0,94,0.99686,3.21,0.58,9.5,6
1197,7.7,0.570,0.21,1.5,0.069,4.0,9,0.99458,3.16,0.54,9.8,6


In [3]:
import numpy as np

colNum = len(df.columns)
# Train - Test is 7 - 3
trainSize = int(len(df) * 0.7)
trainSet = np.zeros(((colNum - 1), trainSize))
trainSet = df[:][0:trainSize]
print('TRAIN SET: \n' + str(trainSet))

testSize = len(df) - trainSize
testSet = np.zeros(((colNum - 1), testSize))
testSet = df[:][trainSize: len(df)]
print('TEST SET:\n' + str(testSet))


TRAIN SET: 
     fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  quality
0              7.4             0.700         0.00  ...       0.56      9.4        5
1              7.8             0.880         0.00  ...       0.68      9.8        5
2              7.8             0.760         0.04  ...       0.65      9.8        5
3             11.2             0.280         0.56  ...       0.58      9.8        6
4              7.4             0.700         0.00  ...       0.56      9.4        5
..             ...               ...          ...  ...        ...      ...      ...
834            8.8             0.685         0.26  ...       0.47      9.4        5
835            7.6             0.665         0.10  ...       0.51      9.3        5
836            6.7             0.280         0.28  ...       0.39     11.7        7
837            6.7             0.280         0.28  ...       0.39     11.7        7
838           10.1             0.310         0.35  ...       0.8

Tạo hàm `linearRegression` để xử lý `trainSet`, ta nhận được 1 model

In [73]:
# dataset: 2D array
# return: trained model
def linearRegression(dataset):
  col = len(dataset[0]) - 1
  # Ax = B
  # x = (A^T*A)^-1 * A^T * B
  A = np.array([i[:col] for i in dataset]) # (839, 11)
  # print(A.shape)
  B = np.array([i[col] for i in dataset]) # (839, 1)
  # print(B.shape)

  x = np.linalg.inv(A.T @ A) @ A.T @ B
  return x

In [74]:
x = linearRegression(np.asarray(trainSet))
x

array([ 9.16934650e-03, -1.03624883e+00, -2.38811108e-01, -1.03481374e-03,
       -1.53280041e+00,  3.39872956e-03, -4.34078032e-03,  4.03200906e+00,
       -3.11991895e-01,  6.71062776e-01,  2.91929462e-01])

Sau khi tìm được model `x`, ta bắt đầu kiểm thử với tập `testSet` <br>
Hàm `predict` trả về kết quả dự đoán 1 dòng dữ liệu dựa trên model <br>
Hàm `validate` kiểm thử toàn bộ dữ liệu và trả về số dòng đoán đúng với sai số nhỏ hơn `tolerance` và sai số trung bình. Nếu không cho `tolerance` giá trị thì mặc định là 0.1 <br>
Kết quả khi test trên file `wine.csv`:
- 52 dự đoán có sai số dưới 0.1
- Sai số trung bình là 0.502

In [5]:
# Ax + By + Cz + ... = quality
def predict(model, data):
  return sum(model * data)

# model: my trained model
# dataset: test set
# return: number of correct, averange tolerance
def validate(model, dataset, tolerance=0.1): 
  total_tolerance = 0
  correct = 0

  col = len(dataset[0]) - 1
  for column in dataset:
    ret = predict(model, column[:col])
    tol = abs(column[col] - ret)

    if tol < tolerance:
      correct += 1
    total_tolerance += tol
    
  return correct, total_tolerance/(len(dataset))

validate(x, np.asarray(testSet))

(52, 0.502440864997521)

## B. Sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất
Lần này, ta sẽ xây dựng model cho từng đặc trưng và tìm đặc trưng cho ra sai số trung bình thấp nhất <br>
Kết quả khi test trên file `wine.csv`: 
- Model có sai số thấp nhất là của cột 'alcohol'
- Đoán đúng 44 lần - Sai số trung bình ~ 0.538

In [71]:
# dataset: 2D array
# return: 
#   + corrects prediction: array
#   + average tolerance: array
def validateEachColumn(train_set, test_set, tolerance=0.1):
  col = len(train_set[0]) - 1 # 11
  train_data = np.array(train_set).T[:col] # (839, 1) for each feature
  train_answer = np.array(train_set).T[col] # (839, 1)
  models = np.zeros((col, 1)) # (11, 1)

  #debug = 'col = ' + str(col) + '\ntrain_data:\n' + str(train_data) + '\ntrain_anwser:\n' + str(train_answer)
  #print(debug)

  # We gonna have 11 models for 11 features
  for i in range(col):
    #print(np.stack((train_data[i], train_answer)))
    dataset = np.stack((train_data[i], train_answer), axis=1) # (839, 2)
    models[i] = linearRegression(dataset) # (1, 1)
    #print(models[i])

  test_data = np.array(test_set).T[:col]
  test_anwser = np.array(test_set).T[col]
  
  corrects = np.zeros((col, 1), dtype=int) # (360, 11)
  avg_tolerances = np.zeros((col, 1)) # (360, 1)
  # Test them
  for i in range(col):
    dataset = np.stack((test_data[i], test_anwser), axis=1) # (360, 2)
    corrects[i], avg_tolerances[i] = validate(models[i], dataset, tolerance)
    print('Correct predictions: ' + str(corrects[i]) + '\tAverage tolerance: ' + str(avg_tolerances[i]))

  return corrects, avg_tolerances


In [72]:
corrects, avg_tolerances = validateEachColumn(np.asarray(trainSet), np.asarray(testSet))

min_column_index = 0
for i in range(len(avg_tolerances)):
  if avg_tolerances[i] == min(avg_tolerances):
    min_column_index = i
    break
df.columns[min_column_index]

Correct predictions: [13]	Average tolerance: [1.20958947]
Correct predictions: [2]	Average tolerance: [2.18464305]
Correct predictions: [7]	Average tolerance: [2.60829562]
Correct predictions: [3]	Average tolerance: [2.47154543]
Correct predictions: [3]	Average tolerance: [2.52078662]
Correct predictions: [5]	Average tolerance: [3.09568422]
Correct predictions: [3]	Average tolerance: [3.51772294]
Correct predictions: [0]	Average tolerance: [0.77646011]
Correct predictions: [11]	Average tolerance: [0.78731701]
Correct predictions: [15]	Average tolerance: [1.17072917]
Correct predictions: [44]	Average tolerance: [0.53865624]


'alcohol'

## C. Xây dựng một mô hình của riêng bạn cho kết quả tốt nhất
Kết quả khảo sát từ câu A cho ra kết quả khá tương đồng với câu B dù cho ta đã sử dụng 11 tính chất thay vì chỉ 1 
- A: Đoán đúng 52 - Sai số TB ~0.502
- B: Đoán đúng 44 - Sai số TB ~0.538 

Ta có thể suy đoán rằng trong 11 tính chất ban đầu, có thể có tính chất không cần thiết thậm chí làm giảm độ chính xác dự đoán

### Phương án 1:
Sử dụng lại model tìm được từ câu A và nhân thêm trọng số. Trọng số này được tính bằng tỷ lệ đoán đúng của cột tương ứng <br>
Ở câu B, với mỗi model, ta đoán đúng tổng cộng 106 lần. Như vậy, trọng số tương ứng với mỗi cột sẽ là tỷ lệ đoán đúng 


In [80]:
total_correct = int(sum(corrects))
print('Total correct: ' + str(total_correct))

Total correct: 106


In [98]:
def modelForSolution1(_corrects, _model):
  _total_correct = sum(_corrects)
  weights = np.asarray([x/_total_correct for x in corrects])
  print('Weights: \n' + str(weights))
  new_model = np.reshape(_model, weights.shape) * weights
  # print(new_model.shape)
  return new_model


In [102]:
new_model = modelForSolution1(np.asarray(corrects), np.asarray(x)) # Remind, x is model found in A. x = (A^T*A)^-1 * A^T * B
print('New model:\n' + str(new_model))

Weights: 
[[0.12264151]
 [0.01886792]
 [0.06603774]
 [0.02830189]
 [0.02830189]
 [0.04716981]
 [0.02830189]
 [0.        ]
 [0.10377358]
 [0.14150943]
 [0.41509434]]
New model:
[[ 1.12454250e-03]
 [-1.95518647e-02]
 [-1.57705449e-02]
 [-2.92871813e-05]
 [-4.33811437e-02]
 [ 1.60317432e-04]
 [-1.22852273e-04]
 [ 0.00000000e+00]
 [-3.23765174e-02]
 [ 9.49617136e-02]
 [ 1.21178267e-01]]
(11, 1)


In [103]:
# Testing time!
validate(np.reshape(new_model, (len(new_model),)), np.asarray(testSet))

(0, 4.665911234504252)

Kết quả:
- Đoán đúng: 0
- Sai số TB: ~4.665

Hóa ra trọng số chỉ khiến cho mọi thứ tệ hơn.

### Phương án 2:
Ta sẽ thử loại bỏ hẳn các cột tệ và chỉ giữ lại cột nào có kết quả tốt (dựa trên câu B) <br>
Cột nào có sai số trung bình nhỏ hơn 1 sẽ được giữ lại

In [114]:
def modelForSolution2(_avg_tolerances, _model):
  weights = np.zeros(_avg_tolerances.shape)
  for i in range(len(_avg_tolerances)):
    if _avg_tolerances[i] < 1:
      weights[i] = 1
  _new_model = weights * np.reshape(_model, weights.shape)
  return _new_model

new_model = modelForSolution2(np.asarray(avg_tolerances), np.asarray(x))
new_model

array([[ 0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [-0.        ],
       [ 0.        ],
       [-0.        ],
       [ 4.03200906],
       [-0.31199189],
       [ 0.        ],
       [ 0.29192946]])

In [115]:
# Testing time!
validate(np.reshape(new_model, (len(new_model),)), np.asarray(testSet))

(28, 0.593639059035015)

Kết quả:
- Đoán đúng: 28
- Sai số TB: ~0.593

Nhận xét: 
- Kết quả tốt hơn rất nhiều so với phương án 1 nhưng vẫn chưa bằng mô hình câu A và B

### Phương án 3:
Ý tưởng giống phương án 2 nhưng thay vì sai số TB > 0, ta giữ lại cột nào đoán đúng hơn 10 lần

In [116]:
def modelForSolution3(_corrects, _model):
  _new_model = np.reshape(_model, _corrects.shape)
  for i in range(len(_corrects)):
    if _corrects[i] < 10:
      _new_model[i] = 0
  return _new_model

new_model = modelForSolution3(np.asarray(corrects), np.asarray(x))
new_model

array([[ 0.00916935],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [-0.31199189],
       [ 0.67106278],
       [ 0.29192946]])

In [117]:
# Testing time!
validate(np.reshape(new_model, (len(new_model),)), np.asarray(testSet))

(0, 3.2623666532647424)

Kết quả:
- Đoán đúng: 0
- Sai số TB: 3.26

Nhận xét:
- Phương án này tồi không kém gì Phương án 1

### Kết luận:
Nếu không lấy mô hình câu A hay B thì phương án 2 là lựa chọn ổn nhất